## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import time
import numpy as np
import pandas as pd
import codecs
import lxml
import csv
import os.path

## Constante variaible

In [35]:
urls_txt = "https.txt"
folder = 'Task 1.2/dataset'
folder2 = 'Task 1.3/anime_tsv'
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36",
    'accept': "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    'referer': "https://myanimelist.net/"
} 
save_path = 'simplePath'

## Q 1.1

In [4]:
c = 0
reg = r'href=\"https://[\w\W]*" id'
htt = list()
for i in range(400):
    cnt = requests.get("https://myanimelist.net/topanime.php?limit="+str(i*50))
    soup = BeautifulSoup(cnt.content, features="lxml")
    links = soup.find_all('h3')
    for l in range(len(links)-3):
        li = links[l].find('a')
        m = re.findall(reg, str(li))
        s = m[0][6:][:-4]+'\n'
        htt.append(s)
f = open("urls_txt", 'w')
for i in htt:
    f.write(i)
f.close

<function TextIOWrapper.close()>

## Q 1.2

In [96]:
def htmls_by_urls(urls_txt, folder):
     # urls_txt: string 'https.txt' from previous task
    # folder: string; eg '/Users/anton/Desktop/ADM/Homework3/html'
    
    with open(urls_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # list of urls
    list_txt = [line.strip() for line in lines]
    
    i = 0
    
    while i < len(list_txt):
        url = list_txt[i]
        # folder where we save html
        al_folder = '{}/page_{}/{}.html'.format(folder, i//50 +1, i+1)
        # download html
        html = requests.get(url, headers)
        if(html.status_code != 200) : 
            time.sleep(120)
            print('error', html.status_code)
        else:
            i += 1
            with open(al_folder, 'w', encoding='utf-8') as g:
                g.write(html.text)


In [97]:
htmls_by_urls(urls_txt, folder)

error 403
error 403
error 403
error 403


## Q 1.3

In [30]:
arr = os.listdir(folder)
fiarr = os.listdir(folder+'/'+arr[0])
fiarr.remove('.ipynb_checkpoints')
for i in range(len(fiarr)):
    fiarr[i] = arr[0]+'/'+fiarr[i]
    

In [31]:
reg1 = r'\n[\w\W]*to'
reg2 = r'to[\w\W]*\n'
animeTitle = []
animeTypes = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []
bs = 9
for fiName in fiarr:
    # anime titles
    path = folder+'/'+fiName
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left sied of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # anime types
            if span.contents[0] == 'Type:':
                animeTypes.append(div.find_all('a')[0].contents[0])
            # anime number of episodes
            if span.contents[0] == 'Episodes:':
                try :
                    animeNumEpisode.append(int(div.contents[2]))
                except:
                    animeNumEpisode.append(0)
            # anime release and end dates
            if span.contents[0] == 'Aired:':
                if len(div.contents[2]) > 21:
                    if "?" in re.findall(reg2, str(div.contents[2]))[0][3:-1]:
                        endDate.append("?")
                    else:
                        releaseDate.append(pd.to_datetime(re.findall(reg1, str(div.contents[2]))[0][2:-3]))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][2:-2]))
                    endDate.append('-')
                    
    # middle side of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        # anime number of members
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        # anime score
        score = div.find_all("div", {"class": "score-label score-"+str(bs)})
        while len(score) == 0:
            bs -= 1
            score = div.find_all("div", {"class": "score-label score-"+str(bs)})
            if bs == 5:
                bs = 10
        animeScore.append(float(score[0].contents[0]))
        # anime number of users
        users = div.find_all("div", {"class": "fl-l score"})
        animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        # anime rank
        rank = div.find_all("span", {"class": "numbers ranked"})
        animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        # anime popularity
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
        
    # anime description
    description = soup.find_all("p", {"itemprop": "description"})
    for br in description[0].find_all("br"):
        br.replace_with("\n")
    animeDescription.append(description[0].contents)
    
    # related animes 
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    x = []
    y = []
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            t = td[i+1].find_all("a")
            y.append(t[0].contents[0])
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
        
    # anime characters and voices
    characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
    chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
    x = []
    y = []
    voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
    for i in chars:
        x.append(i.contents[0].contents[0])
    for i in voices:
        y.append(i.contents[1].contents[0])
    animeCharacters.append(x)
    animeVoices.append(y)
    
    # anime staff
    try:
        staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        staff = staff[1].find_all("td")
        x = []
        y = []
        for i in range(1, len(staff), 2):
            x.append(staff[i].contents[1].contents[0])
            y.append(staff[i].find_all("small")[0].contents[0])
        animeStaff.append([list(i) for i in list(zip(x,y))])
    except:
        print(path)

Task 1.2/dataset/page_1/23.html


In [32]:
df = pd.DataFrame(
    [animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
    index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
df[:5]

,Title,Type,Episodes,Release date,End date,Members,Score,Users,Rank,Popularity,Description,Related,Characters,Voices,Staff
0,Fullmetal Alchemist: Brotherhood,TV,64,2009-04-05,-,2676639,9.16,1622384,1,3,[After a horrific alchemy experiment goes wron...,"[Adaptation: Fullmetal Alchemist, Alternative ...","[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...","[[Cook, Justin, Producer], [Yonai, Noritomo, P..."
1,Gintama: The Final,Movie,1,2021-01-08,-,79486,9,29979,10,1924,"[New , [Gintama], movie.]","[Adaptation: Gintama, Prequel: Gintama.: Shiro...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Ishida, Akira, Hino, Satoshi]","[[Fujita, Youichi, Director], [Miyawaki, Chizu..."
2,Gintama.,TV,12,2017-01-09,-,246290,8.98,108581,11,726,[After joining the resistance against the baku...,"[Adaptation: Gintama, Prequel: Gintama°, Seque...","[Sakata, Gintoki, Kagura, Katsura, Kotarou, Ta...","[Sugita, Tomokazu, Kugimiya, Rie, Ishida, Akir...","[[Fujita, Youichi, Director], [Miyawaki, Chizu..."
3,3-gatsu no Lion 2nd Season,TV,22,2017-10-14,-,324393,8.97,155163,12,529,"[Now in his second year of high school, Rei Ki...","[Adaptation: 3-gatsu no Lion, Prequel: 3-gatsu...","[Kiriyama, Rei, Kawamoto, Hinata, Kawamoto, Ak...","[Kawanishi, Kengo, Hanazawa, Kana, Kayano, Ai,...","[[Shinbou, Akiyuki, Director, Series Compositi..."
4,Koe no Katachi,Movie,1,2016-09-17,-,1780070,8.97,1208990,13,23,"[As a wild youth, elementary school student Sh...","[Adaptation: Koe no Katachi, Other: Koe no Kat...","[Nishimiya, Shouko, Ishida, Shouya, Nishimiya,...","[Hayami, Saori, Irino, Miyu, Yuuki, Aoi, Kanek...","[[Senami, Riri, Assistant Producer], [Yamada, ..."


In [36]:
#For each row I created a tsv file with the value of each anime 
for i in range(len(df)):
    with open(folder2+'/anime_'+str(i)+'.tsv', 'wt') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in df.columns]) #the header row
        tsv_writer.writerow(x for x in df.iloc[i]) #the value under each columns
        